In [15]:
import pandas as pd
import urllib, urllib.request



In [3]:
url = 'http://export.arxiv.org/api/query?search_query=all&start=0&max_results=10'
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))

<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%26id_list%3D%26start%3D0%26max_results%3D10" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all&amp;id_list=&amp;start=0&amp;max_results=10</title>
  <id>http://arxiv.org/api/EzVVceYFgffpe1pN1SaekBDSOE0</id>
  <updated>2023-05-30T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">326165</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">10</opensearch:itemsPerPage>
  <entry>
    <id>http://arxiv.org/abs/1911.11405v1</id>
    <updated>2019-11-26T08:49:54Z</updated>
    <published>2019-11-26T08:49:54Z</published>
    <title>A Note on Computational Complexity of Kill-all Go</title>
    <summ

In [1]:
import feedparser
import urllib
import time 

base_url = 'http://export.arxiv.org/api/query?'

# Search parameters
search_query = 'all'  # search in all categories
start = 0             # start at the first result
total_results = 1000  # number of results to retrieve for each request

current_result = start

query = f'search_query={search_query}&start={current_result}&max_results={total_results}'
response = urllib.request.urlopen(base_url+query).read()




In [13]:
data = []

# Parsing the response
feed = feedparser.parse(response)
for entry in feed.entries:
    data.append({
        "arXiv-id": entry.id.split('/abs/')[-1],
        "Title": entry.title,
        "Summary": entry.summary,
        "Published": entry.published,
        "Updated": entry.updated,
        "Authors": [author.name for author in entry.authors],
        "Subject": [cat.term for cat in entry.tags]
    })

# Move on to the next results
# current_result += total_results

# Respectful crawling by sleeping
# print("Sleeping for a while...")
# time.sleep(15)

In [4]:
feed.entries[0]

{'id': 'http://arxiv.org/abs/1911.11405v1',
 'guidislink': True,
 'link': 'http://arxiv.org/abs/1911.11405v1',
 'updated': '2019-11-26T08:49:54Z',
 'updated_parsed': time.struct_time(tm_year=2019, tm_mon=11, tm_mday=26, tm_hour=8, tm_min=49, tm_sec=54, tm_wday=1, tm_yday=330, tm_isdst=0),
 'published': '2019-11-26T08:49:54Z',
 'published_parsed': time.struct_time(tm_year=2019, tm_mon=11, tm_mday=26, tm_hour=8, tm_min=49, tm_sec=54, tm_wday=1, tm_yday=330, tm_isdst=0),
 'title': 'A Note on Computational Complexity of Kill-all Go',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': '',
  'value': 'A Note on Computational Complexity of Kill-all Go'},
 'summary': 'Kill-all Go is a variant of Go in which Black tries to capture all white\nstones, while White tries to survive. We consider computational complexity of\nKill-all Go with two rulesets, Chinese rules and Japanese rules. We prove that:\n(i) Kill-all Go with Chinese rules is PSPACE-hard, and (ii) Kill-all Go with\n

In [16]:
pd.DataFrame(data)

,arXiv-id,Title,Summary,Published,Updated,Authors,Subject
0,1911.11405v1,A Note on Computational Complexity of Kill-all Go,Kill-all Go is a variant of Go in which Black ...,2019-11-26T08:49:54Z,2019-11-26T08:49:54Z,[Zhujun Zhang],[cs.CC]
1,1210.7708v1,Landau--Kolmogorov inequality revisited,The Landau-Kolmogorov problem consists of find...,2012-10-29T16:12:41Z,2012-10-29T16:12:41Z,[Alexei Shadrin],[math.NA]
2,1602.00450v1,All-in-all-out magnetic domain wall conduction...,"Pyrochlore oxides possessing ""all-in-all-out"" ...",2016-02-01T09:53:30Z,2016-02-01T09:53:30Z,"[T. C. Fujita, M. Uchida, Y. Kozuka, W. Sano, ...",[cond-mat.mtrl-sci]
3,1612.01842v2,An Improved One-to-All Broadcasting in Higher ...,"Recently, a higher dimensional Eisenstein-Jaco...",2016-12-06T15:01:47Z,2016-12-07T06:10:53Z,[Zaid Hussain],[cs.DC]
4,2208.03553v1,The provability logic of all provability predi...,We prove that the provability logic of all pro...,2022-08-06T18:00:01Z,2022-08-06T18:00:01Z,[Taishi Kurahashi],[math.LO]
...,...,...,...,...,...,...,...
995,hep-ph/0208189v3,The Pinch Technique to All Orders,The generalization of the pinch technique to a...,2002-08-21T10:10:39Z,2003-01-14T16:48:42Z,"[D. Binosi, J. Papavassiliou]",[hep-ph]
996,hep-ph/0302018v2,Saturation of optimality limits in hadron-hadr...,In this paper the optimal unitarity lower boun...,2003-02-04T05:28:28Z,2003-02-09T05:21:14Z,"[D. B. Ion, M. L. Ion]",[hep-ph]
997,hep-ph/0602006v1,Proof of the multi-Regge form of QCD amplitude...,The multi--Regge form of QCD amplitudes with g...,2006-02-01T15:41:01Z,2006-02-01T15:41:01Z,"[V. S. Fadin, R. Fiore, M. G. Kozlov, A. V. Re...",[hep-ph]
998,hep-ph/0607064v1,All-order Finiteness of the Higgs Boson Mass i...,"In the dynamical gauge-Higgs unification, it i...",2006-07-06T10:47:48Z,2006-07-06T10:47:48Z,[Yutaka Hosotani],"[hep-ph, hep-th]"


In [6]:
feed.entries[0].authors

[{'name': 'Zhujun Zhang'}]

In [11]:
feed.entries[0].tags

[{'term': 'cs.CC', 'scheme': 'http://arxiv.org/schemas/atom', 'label': None}]

In [73]:
import sqlite3
conn = sqlite3.connect('arxiv_metadata.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE IF NOT EXISTS metadata
             (arxiv_id TEXT, title TEXT, summary TEXT, published TEXT, updated TEXT, authors TEXT)''')